## Init

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../") 

import random
myseed = 13
random.seed(myseed)

import pandas as pd
import numpy as np
import es
import es.data_config
import es.runner.scenario_runner
from es.runner.view_models import show_cube_grid, SpectrumViewModel, show_spectrum_grid
import es.elections
import es.utils
from es.data.movielens_data import *
import bokeh
import bokeh.plotting
from bokeh.io import output_notebook
output_notebook()
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('display.max_rows', 40)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


In [ ]:
%%time
# idata = es.data.movielens_data.MovielensData(es.data_config.data_movielens_25m)
idata = es.data.movielens_data.MovielensData(es.data_config.data_movielens_1m)
iframes = es.elections.InstacartDataFrames.from_data(idata)

## Parameters

In [ ]:
PARAMS = es.utils.dotdict(dict(
        exp_name='tmp',
        k=10,
        searchterms=[        
#             'Hot Shots! (1991)',
            'Star Trek: Generations (1994)', 
#             'Star Trek V: The Final Frontier (1989)',
        ],
        filtering=dict(sample_orders=None),
        rules=[
            "HUV_0", 
#             "sa.HUV_0", 
#             "HUV_1", "sa.HUV_1", 
#             "HUV_2", "sa.HUV_2", 
#             "HUV_3", "sa.HUV_3", 
        ], # prefix sa. means 'use simulated annealing'
        anneal_settings= {'tmax': 8100.0, 'tmin': 2.6, 'steps': 20000, 'updates': 10}, # anneal_settings={'minutes': 0.5},
        scorer_metrics=['tfidfto2'],
        scorer_funcs = [
            es.scorers.AgentSpreadScoringStrategy.from_found_df_sorted_nosearchterm,
        ]
    ))
PARAMS.searchterms = [t.replace('(', '.').replace(')', '.') for t in PARAMS.searchterms]

## Runner

In [ ]:
%%time
runner = es.runner.scenario_runner.ScenarioRunner(iframes, PARAMS)
exp_folder = runner.run(verbose=0)

## Visualisation

In [ ]:
# %%time
# es.runner.view_models.show_spectrum_grid(SpectrumViewModel(), PARAMS, exp_folder, plot_side=500, plot_height=50, max_div_value=150)

## Tfidf search

In [ ]:
# product_name_re = PARAMS.searchterms[0]
# found_df = runner.utils.calc_found_df(runner.utils.find_products(product_name_re, min_order_count=1))
# display(found_df.sort_values("tfidfto2", ascending=False)[:10])

# Analysis

In [ ]:
from es import *
u = es.electionutils.ElectionUtils(iframes)

In [ ]:
rdb = u.load_results_db('results/tmp')
rdb.query("""
select r.rank, r.scorer, r.search_term, r.product_name, t.rank as t_rank, t.order_count as global_count, t.tf, t.idfto2
from results r inner join tfidfs t on r.product_id = t.product_id and r.search_term = t.search_term
where 
r.search_term like '%Generations%'
and
scorer in ('AgentSpread(tfidfto2)', 'AgentCountSqrtSpread(tfidfto2)')
""")
# .to_csv("CountSqrtSpread.tsv", sep='\t', index=False)

## SQL

In [ ]:
from pandasql import sqldf
sql = lambda q: sqldf(q, globals())

products = u.product_search
sql(f"select * from products where product_name like 'Star Trek%' and order_count > 1 order by order_count desc")

In [ ]:
rdb.query("select * from tfidfs where rank>0 and search_term like '%Gener%' and order_count>0")[:30]

In [ ]:
approvals = rdb.query("select order_count from tfidfs where rank>0 and search_term like '%Hot%'").order_count.to_list()